# Удаление ненужных столбцов и поиск координат конкретных адресов через Google Maps

In [ ]:
from selenium import webdriver
from tqdm import tqdm_notebook as tqdmn
import pandas as pd
import folium

lst = [
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201020_182616.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201021_185504.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201111_223641.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201118_223519.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201119_230826.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201123_210620.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201125_032739.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201125_225552.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201126_190637.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201130_210949.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201201_165823.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201202_191224.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201208_202137.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201215_221946.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201216_183747.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201217_144624.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20201224_012524.csv",#
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/Dataset_CVRPTW/20210105_235706.csv"#
]
for i in range(len(lst)):
    
    file_name = lst[i]
    acnc = pd.read_csv(file_name, sep= "\t", keep_default_na=False)
    mel = acnc[
        ['OrderID', 'Weight', 'PickupAddress', 'DeliveryAddress', 'DeliveryTimeRange']].copy()
    mel['Url_depo'] = ['https://www.google.com/maps/search/' + i for i in mel['PickupAddress'] ]
    mel['Url_clients'] = ['https://www.google.com/maps/search/' + i for i in mel['DeliveryAddress'] ]
    Url_With_Coordinates_depo = []
    Url_With_Coordinates_clients = []

    option = webdriver.ChromeOptions()
    prefs = {'profile.default_content_setting_values': {}}
    option.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome("chromedriver.exe", options=option)

    for url in tqdmn(mel.Url_depo, leave=False):
        driver.get(url)
        Url_With_Coordinates_depo.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

    for url in tqdmn(mel.Url_clients, leave=False):
        driver.get(url)
        Url_With_Coordinates_clients.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

    driver.close()

    mel['Url_With_Coordinates_depo'] = Url_With_Coordinates_depo
    mel['Url_With_Coordinates_clients'] = Url_With_Coordinates_clients
    mel = mel[mel.Url_With_Coordinates_clients.str.contains('&zoom=')].copy()
    
    mel['x_depo'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates_depo'] ]
    mel['y_depo'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates_depo'] ]
    mel['x_client'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates_clients'] ]
    mel['y_client'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates_clients'] ]
    

    mel = mel.drop('OrderID', 1)
    mel = mel.drop('PickupAddress', 1)
    mel = mel.drop('DeliveryAddress', 1)
    mel = mel.drop('Url_depo', 1)
    mel = mel.drop('Url_clients', 1)
    mel = mel.drop('Url_With_Coordinates_depo', 1)
    mel = mel.drop('Url_With_Coordinates_clients', 1)

    mel.to_csv(file_name, sep='\t')

# Удаление двух столбцов с координатами депо и добавление координат, веса и временного окна депо в последнюю строку

In [ ]:
for i in range(len(lst)):
    file_name = lst[i]
    acnc = pd.read_csv(file_name, sep= "\t", keep_default_na=False)
    str_data = {'Weight': 0.0, 'DeliveryTimeRange': acnc['DeliveryTimeRange'][0], 'x_client':acnc['x_depo'][0], 
                'y_client': acnc['y_depo'][0]}
    acnc = acnc.drop('x_depo', 1)
    acnc = acnc.drop('y_depo', 1)
    acnc = acnc.drop(acnc.columns[0], axis=1)
    acnc = acnc.append(str_data, ignore_index=True)
    print(acnc)
    acnc.to_csv(file_name, sep='\t')


# Перестановка столбцов в порядке: координаты, вес груза, временное окно

In [ ]:
for i in range(len(lst)):
    file_name = lst[i]
    acnc = pd.read_csv(file_name, sep= "\t", keep_default_na=False)
    acnc = acnc.reindex(columns=['x_client', 'y_client', 'Weight', 'DeliveryTimeRange'])
    acnc.to_csv(file_name, sep='\t', index = False)
acnc

# Чистка координат не удовлетворящих данным широте и долготе

In [ ]:
for i in range(len(lst)):
    file_name = lst[i]
    acnc = pd.read_csv(file_name, sep= "\t", keep_default_na=False)
    acnc = acnc.drop(acnc[(acnc.x_client < 43) | (acnc.x_client >= 44)].index)
    acnc = acnc.drop(acnc[(acnc.y_client < 76) | (acnc.y_client >= 77)].index)
    acnc.to_csv(file_name, sep='\t', index = False)